In [ ]:
import geopandas as gpd
from ipyfilechooser import FileChooser
import numpy as np
from pathlib import Path
import rioxarray as rxr
import rioxarray.merge
from shapely.geometry import mapping
from tqdm.auto import tqdm

from skimage.color import label2rgb
from skimage.segmentation import felzenszwalb

import matplotlib.pyplot as plt

### Select the chip directory

In [ ]:
print("Select the chip directory containing the preprocessed Planet data, Chip shapefile, and clipped Pekel occurance data")
fc = FileChooser(Path.cwd())
display(fc)

In [ ]:
chip_dir = Path(fc.selected_path)

shape_path = list(chip_dir.rglob('*.shp'))
if len(shape_path) < 1:
    raise FileNotFoundError
elif len(shape_path) > 1:
    raise Exception("Multiple shapefiles found")
else:
    shape_path = shape_path[0]

planet_path = list(chip_dir.rglob('*_AnalyticMS_*.tif'))
if len(planet_path) < 1:
    raise FileNotFoundError
elif len(planet_path) > 1:
    raise Exception("Multiple Planet data files found")
else:
    planet_path = planet_path[0]
    
pekel_path = list(chip_dir.rglob('occurrence_*.tif'))
if len(pekel_path) < 1:
    raise FileNotFoundError
elif len(pekel_path) > 1:
    raise Exception("Multiple pekel occurrence files found")
else:
    pekel_path = pekel_path[0]
    
print(shape_path)
print(planet_path)
print(pekel_path)

### Load the chip geometry from the chip shapefile

In [ ]:
shape_path = Path(fc.selected)
shape_path

In [ ]:
geometry_gdf = gpd.read_file(shape_path)
geometry = geometry_gdf['geometry'][0]
geometry

### Collect the chip ID

In [ ]:
chip_id = chip_dir.name
chip_id

### Load Planet data as xarray.DataArray

In [ ]:
chip_image = rxr.open_rasterio(planet_path,
                               masked=True).squeeze()
chip_image

**Write a segmentation function using skimage.segmentation.felzenwalb**

In [ ]:
def get_segmentation(image, min_size):
    image_seg = image.copy()
    
    # So that np.nans don't give us problems
    image_seg = image_seg.fillna(-10_000)

    segments_fz = felzenszwalb(image_seg,
                               # may want to play with this
                               scale=5, 
                               # normally gaussian filter is applied - can experiment
                               sigma=0.,
                               # minimum size of segments
                               min_size=min_size
                              )
    return segments_fz

**Reorder the bands for skimage and generate segmentations with 3 different minimum segment sizes (10, 25, and 50 pixels)**

In [ ]:
chip_image_reshape = chip_image.transpose('y', 'x', 'band')
print(chip_image_reshape.shape)

get_segmentation_partial = lambda min_size: get_segmentation(chip_image_reshape, min_size)
segmentations = list(map(get_segmentation_partial, tqdm([10, 25, 50])))

**

In [ ]:
seg = 1
mask = chip_image == np.nan
X = segmentations[seg].copy()
X[mask[0]] = 0

superpixel_labels_viz = label2rgb(X, bg_label=0)
plt.imshow(superpixel_labels_viz)

In [ ]:
segments_fz = segmentations[seg]
segments_fz

In [ ]:
import rasterio as rio

with rio.open(planet_path) as tif:
    transform = tif.transform
    crs = tif.crs
    profile = tif.profile
    bounds = tif.bounds

In [ ]:
from rscube.rio_tools import get_geopandas_features_from_array

features = get_geopandas_features_from_array(segments_fz.astype(np.int32), 
                                             transform, 
                                             label_name='label')

In [ ]:
df_segments = gpd.GeoDataFrame.from_features(features)
df_segments.head()

In [ ]:
df_segments.crs = crs
df_segments.to_file(chip_dir/'segments.geojson', driver='GeoJSON')

### Generate Features

In [ ]:
pixel_features = chip_image_reshape.stack(z=('y', 'x')).transpose('z', 'band')
pixel_features.shape

In [ ]:
from rscube.nd_tools import get_superpixel_means_as_features

multi_superpixel_features_mean = [get_superpixel_means_as_features(seg, chip_image_reshape) for seg in tqdm(segmentations)]

In [ ]:
from rscube.nd_tools import get_superpixel_stds_as_features

multi_superpixel_features_std = [get_superpixel_stds_as_features(seg, chip_image_reshape) for seg in tqdm(segmentations)]

In [ ]:
from rscube.nd_tools import (get_array_from_features, 
                             get_features_from_array, 
                             get_superpixel_area_as_features, 
                             get_superpixel_means_as_features,
                             get_superpixel_stds_as_features, 
                             scale_img)

multi_superpixel_means = [get_array_from_features(seg, feature) 
                          for (seg, feature) in zip(tqdm(segmentations),
                                                    multi_superpixel_features_mean)]
multi_superpixel_stds = [get_array_from_features(seg, feature) 
                          for (seg, feature) in zip(tqdm(segmentations),
                                                    multi_superpixel_features_std)]

In [ ]:
scale_ind = 0
img_super = multi_superpixel_means[scale_ind]

with rio.open(chip_dir/f'superpixel_means_{scale_ind}_{chip_id}.tif', 'w', **profile) as ds:
    ds.write(img_super.transpose([2, 0, 1]))

In [ ]:
scale_ind = 0
img_super = multi_superpixel_stds[scale_ind]

with rio.open(chip_dir/f'superpixel_stds_{scale_ind}_{chip_id}.tif', 'w', **profile) as ds:
    ds.write(img_super.transpose([2, 0, 1]))

In [ ]:
superpixel_means_pixel_f = [superpixel_means.reshape((-1, chip_image_reshape.shape[-1])) 
                            for superpixel_means in multi_superpixel_means]
superpixel_stds_pixel_f = [superpixel_stds.reshape((-1, chip_image_reshape.shape[-1])) 
                            for superpixel_stds in multi_superpixel_stds]

In [ ]:
all_features = np.hstack(([pixel_features]  
                          + superpixel_means_pixel_f 
                          + superpixel_stds_pixel_f
                         ))

all_features = np.nan_to_num(all_features)

all_features.shape

### Generate Training Data

In [ ]:
pekel_image = rxr.open_rasterio(pekel_path,
                                 masked=True).squeeze()

plt.imshow(pekel_image)
pekel_image

In [ ]:
from scipy import ndimage

OCC_MIN = 8
DIST_TO_EXTENT = 15

water_ind = (pekel_image >= OCC_MIN)
water_extent = (pekel_image > 0)

water_extent_arr = (~water_extent).astype(int)
water_dist = ndimage.distance_transform_edt(water_extent_arr)

land_ind = water_dist > DIST_TO_EXTENT

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(15,5))

ax[0].imshow(water_ind)
ax[0].set_title('Water Mask')

ax[1].imshow(water_dist)
ax[1].set_title('Distance to non-zero occurence')

ax[2].imshow(land_ind)
ax[2].set_title('Land Mask')

In [ ]:
training_data_4326 = np.zeros(pekel_image.shape)
training_data_4326[land_ind] = 2
training_data_4326[water_ind] = 1
plt.title('0 = no label/nodata, 1=water, 2 = land')
plt.imshow(training_data_4326)

In [ ]:
with rio.open(pekel_path) as tif:
    pekel_transform = tif.transform
    pekel_crs = tif.crs
    pekel_profile = tif.profile
    pekel_bounds = tif.bounds
    
print(pekel_crs)


In [ ]:
from dem_stitcher.rio_tools import reproject_arr_to_match_profile

p_temp = pekel_profile.copy()
p_temp['dtype'] = 'float32'
p_temp['nodata'] = None
p_temp['nodata'] = np.nan

training_data_r, p_r = reproject_arr_to_match_profile(training_data_4326, p_temp, profile)
# rounding then casting makes sure the class assigned is closest after reprojection
training_data_r = training_data_r[0, ...]
training_data_r = np.nan_to_num(training_data_r)

plt.imshow(training_data_r)

In [ ]:

with rio.open('test_train.tif', 'w', **p_r) as ds:
    ds.write(training_data_r, 1)

In [ ]:
labels = training_data_r.copy()

In [ ]:
labels_f = labels.reshape((-1, 1)).ravel()

f_mask = ~(labels_f == 0).ravel()
f_mask.shape

In [ ]:
X_labeled_all = all_features[f_mask]
y_labeled_all = labels_f[f_mask]

In [ ]:
np.random.seed(0)
water_ind_f = np.where(y_labeled_all == 1)[0]
land_ind_f = np.where(y_labeled_all == 2)[0]
n = water_ind_f.shape[0]
land_ind_f_sample = np.random.choice(land_ind_f, size=n)

indices_for_labeling = np.hstack([water_ind_f, land_ind_f_sample])
indices_for_labeling.shape

y_labeled = y_labeled_all[indices_for_labeling].astype(int)
X_labeled = X_labeled_all[indices_for_labeling, ...]

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_labeled, 
                                                    y_labeled, 
                                                    test_size=0.2,
                                                    train_size=0.8,
                                                    random_state=0,
                                                    stratify=y_labeled
                                                   )

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=500,
                            oob_score=True,
                            random_state=0,
                            n_jobs=8)

In [ ]:
%%time

rf.fit(X_train, 
       y_train)

In [ ]:
%%time

y_pred = rf.predict(X_test)

In [ ]:
class_dict = {1: 'water', 
              2: 'not_water'}

In [ ]:
import pandas as pd

y_pred_str = pd.Series([class_dict[class_id] for class_id in y_pred], name='Predicted')
y_true_str = pd.Series([class_dict[class_id] for class_id in y_test], name='True')

In [ ]:
df = pd.crosstab(y_true_str, y_pred_str)
df

In [ ]:
profile

In [ ]:
mask = chip_image == profile['nodata']
all_mask_f = mask[1].stack(z=['y', 'x'])

In [ ]:
X_all = all_features[~all_mask_f, 
                     ...]

In [ ]:
%%time

y_all = rf.predict(X_all)

In [ ]:
y_arr = np.zeros(mask[1].shape)

y_arr[~mask[0]] = y_all
y_arr[mask[0]] = 0

In [ ]:
p = profile.copy()
p['count'] = 1 
p['dtype'] = np.uint8

class_path = chip_dir/f'class_out_{chip_id}.tif'
with rio.open(class_path, 'w', **p) as ds:
    ds.write(y_arr.astype(np.uint8), 1)

In [ ]:
plt.imshow(y_arr, interpolation='none')

In [ ]:
class_img = rxr.open_rasterio(class_path,
                                 masked=True).squeeze()
class_img_clipped = class_img.rio.clip(geometry_gdf.geometry.apply(mapping), 'EPSG:4326')
plt.imshow(class_img_clipped)

In [ ]:
class_img_clipped = class_img_clipped.fillna(255)
class_img_clipped = class_img_clipped.where(np.logical_or(class_img_clipped == 255, class_img_clipped == 1), 0)
class_img_clipped = class_img_clipped.astype('uint8')
class_img_clipped = class_img_clipped.assign_attrs({'_FillValue': 255})
class_img_clipped.rio.to_raster(class_path)

In [ ]:
from osgeo import gdal

ds = gdal.Open(str(class_path), 1)
band = ds.GetRasterBand(1)

# create color table
colors = gdal.ColorTable()

# set color for each value
colors.SetColorEntry(0, (255, 255, 255))
colors.SetColorEntry(1, (14, 148, 237))
colors.SetColorEntry(255, (0, 0, 0))


# set color table and color interpretation
band.SetRasterColorTable(colors)
band.SetRasterColorInterpretation(gdal.GCI_PaletteIndex)

# close and save file
del band, ds

In [ ]:
class_img_clipped